In [63]:
import pandas as pd
#matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

covid_data = pd.read_csv('google_new3.csv',sep=";")

In [64]:
str_column =['iso','country']
target_column = ['total_cases','fatalities'] 
predictors = list(set(list(covid_data.columns))-set(target_column)-set(str_column))
covid_data[predictors] = covid_data[predictors]
print(covid_data.columns)
covid_data.describe().transpose()

Index(['iso', 'country', 'date', 'grocery_pharmacy', 'parks', 'residential',
       'retail_recreation', 'transit_stations', 'workplaces', 'total_cases',
       'fatalities'],
      dtype='object')


,count,mean,std,min,25%,50%,75%,max
date,817.0,22.000000,12.417275,1.000,11.000,22.000,33.000,43.000
grocery_pharmacy,817.0,-8.229424,21.313369,-84.798,-16.578,0.088,4.838,59.474
parks,817.0,-13.394035,32.891636,-91.456,-32.841,-6.498,6.961,75.002
residential,817.0,9.545845,10.545377,-3.647,0.744,5.912,16.695,40.343
retail_recreation,817.0,-26.205257,30.992728,-95.158,-49.975,-14.427,-0.632,19.699
transit_stations,817.0,-28.370122,30.211025,-88.956,-54.145,-19.502,-0.970,16.822
workplaces,817.0,-19.126192,25.983021,-79.293,-40.582,-8.395,1.037,21.347
total_cases,817.0,10699.773562,31656.464952,0.000,27.000,648.000,4585.000,336912.000
fatalities,817.0,566.007344,2008.520488,0.000,0.000,6.000,79.000,15887.000


In [85]:
ar_covid_data = covid_data.loc[(covid_data['country'] == 'Argentina')]
au_covid_data = covid_data.loc[(covid_data['country'] == 'Australia')]
br_covid_data = covid_data.loc[(covid_data['country'] == 'Brazil')]
ca_covid_data = covid_data.loc[(covid_data['country'] == 'Canada')]
de_covid_data = covid_data.loc[(covid_data['country'] == 'Germany')]
es_covid_data = covid_data.loc[(covid_data['country'] == 'Spain')]
fr_covid_data = covid_data.loc[(covid_data['country'] == 'France')]
gb_covid_data = covid_data.loc[(covid_data['country'] == 'United Kingdom')]
id_covid_data = covid_data.loc[(covid_data['country'] == 'Indonesia')]
in_covid_data = covid_data.loc[(covid_data['country'] == 'India')]
it_covid_data = covid_data.loc[(covid_data['country'] == 'Italy')]
jp_covid_data = covid_data.loc[(covid_data['country'] == 'Japan')]
kr_covid_data = covid_data.loc[(covid_data['country'] == 'South Korea')]
mx_covid_data = covid_data.loc[(covid_data['country'] == 'Mexico')]
sa_covid_data = covid_data.loc[(covid_data['country'] == 'Saudi Arabia')]
se_covid_data = covid_data.loc[(covid_data['country'] == 'Sweden')]
tr_covid_data = covid_data.loc[(covid_data['country'] == 'Turkey')]
us_covid_data = covid_data.loc[(covid_data['country'] == 'US')]
za_covid_data = covid_data.loc[(covid_data['country'] == 'South Africa')]

In [104]:
X=covid_data[predictors]
Y1=covid_data["total_cases"]
Y2=covid_data["fatalities"]

X_train, X_test, y1_train, y1_test = train_test_split(X, Y1, test_size=0.30, random_state=40)
X2_train, X2_test, y2_train, y2_test = train_test_split(X, Y2, test_size=0.30, random_state=40)
print(X_train.shape)
print(X_test.shape)

(571, 7)
(246, 7)


In [114]:
mlp = MLPRegressor(hidden_layer_sizes=(8,8), activation='relu', solver='adam', max_iter=50000)
mlp.fit(X_train,y1_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

In [115]:
print(mean_squared_error(y1_train,predict_train))
print(r2_score(y1_train,predict_train))

192470906.99322206
0.8002801655640032


In [116]:
print(mean_squared_error(y1_test,predict_test))
print(r2_score(y1_test,predict_test))

502387176.4239784
0.5370472675604805


In [145]:

score
##mlp.fit(X2_train,y2_train)
##predict_train = mlp.predict(X2_train)
##predict_test = mlp.predict(X2_test)
##print(mean_squared_error(y2_train,predict_train))
##score=r2_score(y2_train,predict_train)
##print(score)
##print(mean_squared_error(y2_test,predict_test))
##print(r2_score(y2_test,predict_test))

0.6346688189576661